In [1]:
# Main notebook for playing around looking at different things in data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
plt.rcParams['lines.markersize'] = 10
plt.rcParams['lines.linewidth'] = 3
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]


saveFigures is set to: True
Done loading packages


In [2]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath


for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])

print(latestdir)
print(latestDate)

# latestdir = rootdir+ '/SSI_data_2021-04-29'
# latestDate = pd.to_datetime(latestdir[-10:])

# print(latestdir)
# print(latestDate)

d:\Pandemix\Github\DanskeData/ssi_data\SSI_data_2021-05-14
2021-05-14 00:00:00


In [11]:
# Load the data
fulldfPCR = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
fulldf = pd.read_csv(latestdir+'/Test_pos_over_time_antigen.csv',delimiter = ';',dtype=str)

# # Cut out the last two rows of summary numbers
dfPCR = fulldfPCR.iloc[:-2,:].copy()
df = fulldf.iloc[:-2,:].copy()

In [12]:
dfPCR["PosPct"] = pd.to_numeric(dfPCR["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfPCR["Date"] = dfPCR["Date"].astype('datetime64[D]')
df["PosPct"] = pd.to_numeric(df["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
df["Date"] = df["Date"].astype('datetime64[D]')

rows_to_fix_period_in = ["NewPositive","NotPrevPos","PrevPos","Tested","Tested_kumulativ"]
for name in rows_to_fix_period_in:
    df[name] = pd.to_numeric(df[name].astype(str).apply(lambda x: x.replace('.','')))
    dfPCR[name] = pd.to_numeric(dfPCR[name].astype(str).apply(lambda x: x.replace('.','')))


In [13]:
# Load the data

df2 = pd.read_csv(latestdir+'/Antigentests_pr_dag.csv',delimiter = ';')

# # Remove the first two rows which are from april 2020
# df2 = df2.iloc[2:,:].copy()

In [14]:
df2["Dato"] = df2["Dato"].astype('datetime64[D]')
df2["Andel_AGpos_PCRkonf_pct"] = pd.to_numeric(df2["Andel_AGpos_PCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))
df2["Andel_AGminusPCRkonf_pct"] = pd.to_numeric(df2["Andel_AGminusPCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))

In [17]:
# print(df2.columns.values)
# df2.tail()
# df2

In [18]:
# plt.figure()
# # plt.plot(df2.Dato,df2.AG_testede)
# # plt.plot(df2.Dato,df2.Andel_AGpos_PCRkonf_pct)
# plt.plot(df2.Dato,df2.AG_pos)
# # plt.plot(df2.Dato,df2.AGpos_PCRkonf)
# plt.plot(df2.Dato,df2.AGpos_PCRkonf+df2.AGpos_minusPCRkonf)
# # plt.plot(df2.Dato,df2.AGpos_PCRpos)
# # plt.plot(df2.Dato,df2.AGposPCRneg)
# # plt.plot(df2.Dato,df2.AGpos_PCRpos+df2.AGposPCRneg)
# plt.show()

In [20]:
# fig,(ax1,ax2) = plt.subplots(2,1)
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)

ax1.fill_between(df2.Dato,df2.AG_pos,color='blue',label='AG pos, PCR testet')
ax1.fill_between(df2.Dato,df2.AGpos_minusPCRkonf,color='grey',label='AG pos, afventer PCR')

ax2.fill_between(df2.Dato,100*np.ones(df2.Dato.shape),color='xkcd:mint',label='AG pos, PCR neg')
ax2.fill_between(df2.Dato,100*np.divide(df2.AGpos_PCRpos,df2.AGpos_PCRkonf),color='g',label='AG pos, PCR pos')

numAGnegKonf = df2.AGnegPCRpos + df2.AGnegPCRneg

ax3.fill_between(df2.Dato,100*np.ones(df2.Dato.shape),color='xkcd:mint',label='AG neg, PCR neg')
ax3.fill_between(df2.Dato,100*np.divide(df2.AGnegPCRpos,numAGnegKonf),color='g',label='AG neg, PCR pos')

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=100)
ax3.set_ylim(bottom=0,top=3)
ax1.set_xlim([df2.Dato[0],df2.iloc[-1,0]])
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
df.tail()

,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
396,2021-05-09,270,267772,0.1,784,268556,12165562
397,2021-05-10,434,486496,0.1,1426,487922,12653484
398,2021-05-11,446,388566,0.1,1032,389598,13043082
399,2021-05-12,459,419549,0.1,1085,420634,13463716
400,2021-05-13,316,292677,0.1,858,293535,13757251


In [22]:
dfPCR.tail()

,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
468,2021-05-09,798,158180,0.5,4680,162860,28782944
469,2021-05-10,1130,203029,0.6,7285,210314,28993258
470,2021-05-11,1279,189198,0.7,6750,195948,29189206
471,2021-05-12,1134,208907,0.5,6616,215523,29404729
472,2021-05-13,151,31118,0.5,988,32106,29436835


In [23]:
df2.tail()

,Dato,AG_testede,AG_pos,AGpos_PCRkonf,Andel_AGpos_PCRkonf_pct,AGpos_minusPCRkonf,Andel_AGminusPCRkonf_pct,AGpos_PCRpos,AGposPCRneg,AGnegPCRpos,AGnegPCRneg
94,2021-05-06,445188,353,326,92.351275,27,7.648725,203,119,264,24198
95,2021-05-07,404552,387,355,91.731266,32,8.268734,190,156,223,21715
96,2021-05-08,240415,264,248,93.939394,16,6.060606,160,83,189,15463
97,2021-05-09,268669,297,277,93.265993,20,6.734007,170,104,299,17792
98,2021-05-10,488178,476,452,94.957983,24,5.042017,283,163,422,29926


In [24]:
fig,ax1 = plt.subplots(1,1)
ax1.plot(df.Date,100*df.Tested/5800000)
meanWidth=7
ax1.plot(rnTime(df.Date,meanWidth),100*rnMean(df.Tested,meanWidth)/5800000,'k:')

ax1.set_xlim([np.datetime64('2021-02-01'),df.iloc[-1,0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
# fig,ax1 = plt.subplots(1,1)
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.plot(df.Date,df.Tested,label='Antal AG tests')
ax1.plot(dfPCR.Date,dfPCR.Tested,'k*:',label='Antal PCR tests')
ax1.plot(rnTime(dfPCR.Date,7),rnMean(dfPCR.Tested,7),'k',label='Antal PCR tests, 7 dages gennemsnit')

# ax2.plot(df.Date,df.PosPct,label='Positiv procent, AG')
# ax2.plot(dfPCR.Date,dfPCR.PosPct,label='Positive procent, PCR')
ax2.plot(df.Date,100*np.divide(df.NewPositive,df.Tested),label='Positiv procent, AG')
ax2.plot(dfPCR.Date,100*np.divide(dfPCR.NewPositive,dfPCR.Tested),'k',label='Positiv procent, PCR')


ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

ax1.grid()
ax2.grid()

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=0.7)

ax1.set_xlim(left=np.datetime64('2021-02-01'),right=date.today())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d %b\n%Y'))

plt.tight_layout()

# if saveFigures:
#     plt.savefig('figs/AntalTestOgPosPct')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
firstDate = np.datetime64('2021-02-01')
# lastDate = np.datetime64('2021-04-24')
# lastDate = np.datetime64('2021-05-02')
# lastDate = np.datetime64(date.today()) - np.timedelta64(6,'D')
# lastDate = np.datetime64('2021-05-15')
lastDate = np.datetime64(df2.iloc[-1,0],'D') + np.timedelta64(1,'D')

curDates = np.arange(firstDate,lastDate)

# posAG  = df[df.Date.isin(curDates)].NewPositive 
PCRpos = dfPCR[dfPCR.Date.isin(curDates)].NewPositive.values
numPCR = dfPCR[dfPCR.Date.isin(curDates)].Tested.values
AGpos = df[df.Date.isin(curDates)].NewPositive.values
numAG = df[df.Date.isin(curDates)].Tested.values
AGposPCRpos = df2[df2.Dato.isin(curDates)].AGpos_PCRpos.values
# PCRposPct = dfPCR[dfPCR.Date.isin(curDates)].PosPct.values
PCRposPct = np.divide(PCRpos,numPCR)
AGposPct = np.divide(AGpos,numAG)

# Positive PCR-tests, excluding those that only got tested because of a positive AG-test
PCRpos_nonAG = PCRpos-AGposPCRpos

# Factor used for scaling with respect to number of tests
scaleFactor = 0.7

# Screening-ratio (Guess about how many tests would have been carried out in the absence of wide screening)
screenRatio = 1/6


scaleScreenRatio = screenRatio**scaleFactor

# 
multFactor = (1-screenRatio**scaleFactor)


fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.plot(curDates,AGposPCRpos,label='AG positive, PCR confirmed')
ax1.plot(curDates,PCRpos,label='PCR positive, total')
ax1.plot(curDates,PCRpos_nonAG,label='PCR positive, non-AG')
ax1.plot(curDates,np.multiply(PCRpos_nonAG,scaleScreenRatio),label='PCR positive, non-AG\nAssumed due to symptoms')
ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG\nAssumed due to screening')
ax1.plot(curDates,AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG, screening,\nand AG confirmatory tests')

meanWidth = 7
ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),meanWidth),'k:',label='PCR positive, non-AG, screening,\nand AG confirmatory tests, 7 day mean')
# ax1.plot(curDates,PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),label='PCR positive, as neighbours')

foundThroughScreening = AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)
foundThroughPCRScreening = np.multiply(PCRpos_nonAG,multFactor)


ax2.plot(curDates,100*PCRposPct,'b.-',linewidth=0.5,markersize=2,label='PCR, total')
ax2.plot(rnTime(curDates,meanWidth),100*rnMean(PCRposPct,meanWidth),'b',label='PCR, total, '+str(meanWidth)+' day mean')
ax2.plot(curDates,100*AGposPct,'.-',color='xkcd:dark red',label='AG',linewidth=0.5,markersize=2)
ax2.plot(rnTime(curDates,meanWidth),100*rnMean(AGposPct,meanWidth),color='xkcd:dark red',label='AG, '+str(meanWidth)+' day mean')
ax2.plot(curDates,2*100*AGposPct,'.-',color='xkcd:red',label='AG, multiplied by 2',linewidth=0.5,markersize=2)
ax2.plot(rnTime(curDates,meanWidth),2*100*rnMean(AGposPct,meanWidth),color='xkcd:red',label='AG, multiplied by 2, '+str(meanWidth)+' day mean')

ax2.plot(curDates,100*np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),'.-',linewidth=0.5,markersize=2,color='xkcd:green',label='PCR, screening')
ax2.plot(rnTime(curDates,meanWidth),rnMean(100*np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),meanWidth),color='xkcd:green',label='PCR, screening, '+str(meanWidth)+' day mean')
# ax2.plot(curDates,6*100*np.divide(PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),numPCR),label='PCR positive, as neighbours')

# ax2.plot(rnTime(curDates,meanWidth),100*rnMean(PCRposPct,meanWidth),'b')
# ax2.plot(rnTime(curDates,meanWidth),rnMean(6*100*np.divide(PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),numPCR),7),':')

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
ax2.grid()
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/EstimateOfEffectOfScreening')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)

meanWidth = 7

ax1.plot(curDates,AGposPCRpos,'.-',color='xkcd:hot pink',linewidth=0.5,markersize=2,label='AG positive, PCR confirmed')
ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos,meanWidth),color='xkcd:hot pink',label=str(meanWidth)+' day running mean')
ax1.plot(curDates,PCRpos,'.-',color='xkcd:rust',linewidth=0.5,markersize=2,label='PCR positive, total')
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos,meanWidth),color='xkcd:rust',label=str(meanWidth)+' day running mean')
ax1.plot(curDates,PCRpos_nonAG,'g.-',linewidth=0.5,markersize=2,label='PCR positive,\nexcluding tests to confirm AG-tests')
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),'g',label=str(meanWidth)+' day running mean')


ax2.fill_between(curDates,np.multiply(PCRpos_nonAG,multFactor),PCRpos_nonAG,color='g',label='PCR positive,\nexcluding tests to confirm AG-tests')
ax2.fill_between(curDates,np.multiply(PCRpos_nonAG,multFactor),color='xkcd:royal blue',label=f'PCR positive, non-AG\nAssumed due to screening\n({100*multFactor:.2f} %)')

# ax1.plot(curDates,np.multiply(PCRpos_nonAG,scaleScreenRatio),label='PCR positive, non-AG\nAssumed due to symptoms')
# ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG\nAssumed due to screening')
ax3.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),'.-',linewidth=0.5,markersize=2,color='xkcd:royal blue',label=f'PCR positive, non-AG\nAssumed due to screening\n({100*multFactor:.2f} %)')
ax3.plot(rnTime(curDates,meanWidth),rnMean(np.multiply(PCRpos_nonAG,multFactor),meanWidth),'xkcd:royal blue',label=str(meanWidth)+' day running mean')

ax3.plot(curDates,AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),'m.-',linewidth=0.5,markersize=2,label='PCR positive, non-AG, screening,\nand AG confirmatory tests')

ax3.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),meanWidth),'m',label=str(meanWidth)+' day running mean')

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
ax3.grid()
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax3.set_ylim(bottom=0)



ax1.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
plt.tight_layout()


if saveFigures:
    plt.savefig('figs/EstimateOfEffectOfScreeningNew')

<ipython-input-55-7978e356eee4>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)
fig,ax1 = plt.subplots(1,1,sharex=True)

meanWidth = 7

ax1.plot(curDates,PCRpos,'.-',color='xkcd:rust',linewidth=0.5,markersize=2,label='PCR positive, total')
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos,meanWidth),color='xkcd:rust')
ax1.plot(curDates,PCRpos_nonAG,'g.-',linewidth=0.5,markersize=2,label='PCR positive, due to only PCR')
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),'g')
ax1.plot(curDates,AGposPCRpos,'.-',color='xkcd:hot pink',linewidth=0.5,markersize=2,label='PCR positive, due to AG')
ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos,meanWidth),color='xkcd:hot pink')


# ax2.fill_between(curDates,np.multiply(PCRpos_nonAG,multFactor),PCRpos_nonAG,color='g',label='PCR positive,\nexcluding tests to confirm AG-tests')
# ax2.fill_between(curDates,np.multiply(PCRpos_nonAG,multFactor),color='xkcd:royal blue',label=f'PCR positive, non-AG\nAssumed due to screening\n({100*multFactor:.2f} %)')

# # ax1.plot(curDates,np.multiply(PCRpos_nonAG,scaleScreenRatio),label='PCR positive, non-AG\nAssumed due to symptoms')
# # ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG\nAssumed due to screening')
# ax3.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),'.-',linewidth=0.5,markersize=2,color='xkcd:royal blue',label=f'PCR positive, non-AG\nAssumed due to screening\n({100*multFactor:.2f} %)')
# ax3.plot(rnTime(curDates,meanWidth),rnMean(np.multiply(PCRpos_nonAG,multFactor),meanWidth),'xkcd:royal blue',label=str(meanWidth)+' day running mean')

# ax3.plot(curDates,AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),'m.-',linewidth=0.5,markersize=2,label='PCR positive, non-AG, screening,\nand AG confirmatory tests')

# ax3.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),meanWidth),'m',label=str(meanWidth)+' day running mean')

# ax1.legend(loc='center center',bbox_to_anchor = (0.5, 1.0))
ax1.legend(loc='upper left')
# ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
# ax3.grid()
ax1.set_ylim(bottom=0,top=1300)
# ax2.set_ylim(bottom=0)
# ax3.set_ylim(bottom=0)

ax1.set_ylabel('Count')


ax1.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
plt.tight_layout()


if saveFigures:
    plt.savefig('figs/Viggo_Screening')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)
fig,ax1 = plt.subplots(1,1,sharex=True)

meanWidth = 7

ax1.plot(curDates,PCRpos_nonAG,'g.-',linewidth=0.5,markersize=2,label='PCR positive, due to only PCR (all)')
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),'g')


# ax2.fill_between(curDates,np.multiply(PCRpos_nonAG,multFactor),PCRpos_nonAG,color='g',label='PCR positive,\nexcluding tests to confirm AG-tests')
ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),'.-',linewidth=0.5,markersize=2,color='xkcd:royal blue',label=f'PCR positive, due to only PCR (screening, {100*multFactor:.2f} %)')
ax1.plot(rnTime(curDates,meanWidth),rnMean(np.multiply(PCRpos_nonAG,multFactor),meanWidth),color='xkcd:royal blue')

# # ax1.plot(curDates,np.multiply(PCRpos_nonAG,scaleScreenRatio),label='PCR positive, non-AG\nAssumed due to symptoms')
# # ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG\nAssumed due to screening')
# ax3.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),'.-',linewidth=0.5,markersize=2,color='xkcd:royal blue',label=f'PCR positive, non-AG\nAssumed due to screening\n({100*multFactor:.2f} %)')
# ax3.plot(rnTime(curDates,meanWidth),rnMean(np.multiply(PCRpos_nonAG,multFactor),meanWidth),'xkcd:royal blue',label=str(meanWidth)+' day running mean')

# ax3.plot(curDates,AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),'m.-',linewidth=0.5,markersize=2,label='PCR positive, non-AG, screening,\nand AG confirmatory tests')

# ax3.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),meanWidth),'m',label=str(meanWidth)+' day running mean')

# ax1.legend(loc='center center',bbox_to_anchor = (0.5, 1.0))
# ax1.legend()
ax1.legend(loc='upper left')
# ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
# ax3.grid()
ax1.set_ylim(bottom=0,top=1300)
# ax2.set_ylim(bottom=0)
# ax3.set_ylim(bottom=0)

ax1.set_ylabel('Count')


ax1.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
plt.tight_layout()


if saveFigures:
    plt.savefig('figs/Viggo_Screening2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)
fig,ax1 = plt.subplots(1,1,sharex=True)

meanWidth = 7

# ax1.plot(curDates,PCRpos_nonAG,'g.-',linewidth=0.5,markersize=2,label='PCR positive, due to only PCR (all)')
# ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),'g')


# ax2.fill_between(curDates,np.multiply(PCRpos_nonAG,multFactor),PCRpos_nonAG,color='g',label='PCR positive,\nexcluding tests to confirm AG-tests')
ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),'.-',linewidth=0.5,markersize=2,color='xkcd:royal blue',label=f'PCR positive, due to only PCR (screening, {100*multFactor:.2f} %)')
ax1.plot(rnTime(curDates,meanWidth),rnMean(np.multiply(PCRpos_nonAG,multFactor),meanWidth),color='xkcd:royal blue')

ax1.plot(curDates,AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),'m.-',linewidth=0.5,markersize=2,label=f'PCR positive, due to only PCR (screening, {100*multFactor:.2f} %) + AG confirmatory tests')

ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),meanWidth),'m')


# # ax1.plot(curDates,np.multiply(PCRpos_nonAG,scaleScreenRatio),label='PCR positive, non-AG\nAssumed due to symptoms')
# # ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG\nAssumed due to screening')
# ax3.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),'.-',linewidth=0.5,markersize=2,color='xkcd:royal blue',label=f'PCR positive, non-AG\nAssumed due to screening\n({100*multFactor:.2f} %)')
# ax3.plot(rnTime(curDates,meanWidth),rnMean(np.multiply(PCRpos_nonAG,multFactor),meanWidth),'xkcd:royal blue',label=str(meanWidth)+' day running mean')

# ax3.plot(curDates,AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),'m.-',linewidth=0.5,markersize=2,label='PCR positive, non-AG, screening,\nand AG confirmatory tests')

# ax3.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),meanWidth),'m',label=str(meanWidth)+' day running mean')

# ax1.legend(loc='center center',bbox_to_anchor = (0.5, 1.0))
# ax1.legend()
ax1.legend(loc='upper left')
# ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
# ax3.grid()
ax1.set_ylim(bottom=0,top=1300)
# ax2.set_ylim(bottom=0)
# ax3.set_ylim(bottom=0)

ax1.set_ylabel('Count')


ax1.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
plt.tight_layout()


if saveFigures:
    plt.savefig('figs/Viggo_Screening3')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
# firstDate = np.datetime64('2021-02-01')
# lastDate = np.datetime64('2021-04-24')

# curDates = np.arange(firstDate,lastDate)

# posAG  = df[df.Date.isin(curDates)].NewPositive 
PCRpos = dfPCR[dfPCR.Date.isin(curDates)].NewPositive.values
numPCR = dfPCR[dfPCR.Date.isin(curDates)].Tested.values
AGpos = df[df.Date.isin(curDates)].NewPositive.values
numAG = df[df.Date.isin(curDates)].Tested.values
AGposPCRpos = df2[df2.Dato.isin(curDates)].AGpos_PCRpos.values
# PCRposPct = dfPCR[dfPCR.Date.isin(curDates)].PosPct.values
PCRposPct = np.divide(PCRpos,numPCR)
AGposPct = np.divide(AGpos,numAG)

# Positive PCR-tests, excluding those that only got tested because of a positive AG-test
PCRpos_nonAG = PCRpos-AGposPCRpos

# Factor used for scaling with respect to number of tests
scaleFactor = 0.7

# Screening-ratio (Guess about how many tests would have been carried out in the absence of wide screening)
screenRatio = 1/6


scaleScreenRatio = screenRatio**scaleFactor

# 
multFactor = (1-screenRatio**scaleFactor)


fig,ax2 = plt.subplots(1,1,sharex=True)

# ax1.plot(curDates,AGposPCRpos,label='AG positive, PCR confirmed')
# ax1.plot(curDates,PCRpos,label='PCR positive, total')
# ax1.plot(curDates,PCRpos_nonAG,label='PCR positive, non-AG')
# ax1.plot(curDates,np.multiply(PCRpos_nonAG,scaleScreenRatio),label='PCR positive, non-AG\nAssumed due to symptoms')
# ax1.plot(curDates,np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG\nAssumed due to screening')
# ax1.plot(curDates,AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),label='PCR positive, non-AG, screening,\nand AG confirmatory tests')

meanWidth = 7
# ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor),meanWidth),'k:',label='PCR positive, non-AG, screening,\nand AG confirmatory tests, 7 day mean')
# ax1.plot(curDates,PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),label='PCR positive, as neighbours')

foundThroughScreening = AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)
foundThroughPCRScreening = np.multiply(PCRpos_nonAG,multFactor)


# ax2.plot(curDates,100*PCRposPct,'b.-',linewidth=0.5,markersize=2,label='PCR, total')
ax2.plot(rnTime(curDates,meanWidth),100*rnMean(PCRposPct,meanWidth),'b',label='PCR, total')
# ax2.plot(curDates,100*AGposPct,'.-',color='xkcd:dark red',label='AG',linewidth=0.5,markersize=2)
ax2.plot(rnTime(curDates,meanWidth),100*rnMean(AGposPct,meanWidth),color='xkcd:dark red',label='AG')
# ax2.plot(curDates,2*100*AGposPct,'.-',color='xkcd:red',label='AG, multiplied by 2',linewidth=0.5,markersize=2)
ax2.plot(rnTime(curDates,meanWidth),2*100*rnMean(AGposPct,meanWidth),'--',color='xkcd:red',label='AG, multiplied by 2')

# ax2.plot(curDates,100*np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),'.-',linewidth=0.5,markersize=2,color='xkcd:green',label='PCR, screening')
ax2.plot(rnTime(curDates,meanWidth),rnMean(100*np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),meanWidth),color='xkcd:green',label='PCR, screening')
# ax2.plot(curDates,6*100*np.divide(PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),numPCR),label='PCR positive, as neighbours')

# ax2.plot(rnTime(curDates,meanWidth),100*rnMean(PCRposPct,meanWidth),'b')
# ax2.plot(rnTime(curDates,meanWidth),rnMean(6*100*np.divide(PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),numPCR),7),':')

ax2.set_ylabel('Positive percentage [%]')
# ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='upper right')
# ax1.grid()
ax2.grid()
# ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=0.8)
ax2.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/Viggo_pospct')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
# firstDate = np.datetime64('2021-02-01')
# lastDate = np.datetime64('2021-04-24')

# curDates = np.arange(firstDate,lastDate)

# posAG  = df[df.Date.isin(curDates)].NewPositive 
PCRpos = dfPCR[dfPCR.Date.isin(curDates)].NewPositive.values
numPCR = dfPCR[dfPCR.Date.isin(curDates)].Tested.values
AGpos = df[df.Date.isin(curDates)].NewPositive.values
numAG = df[df.Date.isin(curDates)].Tested.values
AGposPCRpos = df2[df2.Dato.isin(curDates)].AGpos_PCRpos.values
# PCRposPct = dfPCR[dfPCR.Date.isin(curDates)].PosPct.values
PCRposPct = np.divide(PCRpos,numPCR)
AGposPct = np.divide(AGpos,numAG)

# Positive PCR-tests, excluding those that only got tested because of a positive AG-test
PCRpos_nonAG = PCRpos-AGposPCRpos

# Factor used for scaling with respect to number of tests
scaleFactor = 0.7

# Screening-ratio (Guess about how many tests would have been carried out in the absence of wide screening)
screenRatio = 1/6


scaleScreenRatio = screenRatio**scaleFactor

# 
multFactor = (1-screenRatio**scaleFactor)


fig,ax2 = plt.subplots(1,1,sharex=True)


meanWidth = 7

foundThroughScreening = AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)
foundThroughPCRScreening = np.multiply(PCRpos_nonAG,multFactor)


# # ax2.plot(curDates,100*PCRposPct,'b.-',linewidth=0.5,markersize=2,label='PCR, total')
# ax2.plot(rnTime(curDates,meanWidth),100*rnMean(PCRposPct,meanWidth),'b',label='PCR, total')
# # ax2.plot(curDates,100*AGposPct,'.-',color='xkcd:dark red',label='AG',linewidth=0.5,markersize=2)
# ax2.plot(rnTime(curDates,meanWidth),100*rnMean(AGposPct,meanWidth),color='xkcd:dark red',label='AG')
# # ax2.plot(curDates,2*100*AGposPct,'.-',color='xkcd:red',label='AG, multiplied by 2',linewidth=0.5,markersize=2)
# ax2.plot(rnTime(curDates,meanWidth),2*100*rnMean(AGposPct,meanWidth),'--',color='xkcd:red',label='AG, multiplied by 2')

# ax2.plot(curDates,100*np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),'.-',linewidth=0.5,markersize=2,color='xkcd:green',label='PCR, screening')
ax2.plot(rnTime(curDates,meanWidth),5800000*rnMean(np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),meanWidth),color='xkcd:green',label='PCR, screening')
# ax2.plot(curDates,6*100*np.divide(PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),numPCR),label='PCR positive, as neighbours')

# ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='upper right')
# ax1.grid()
ax2.grid()
# ax2.set_ylim(bottom=0)
# ax2.set_ylim(bottom=0,top=0.8)
ax2.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/Viggo_pospct2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
# Positiv percentage of estimate
# First show false negative rate
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

# ax2.plot(curDates,100*PCRposPct,'b.-',linewidth=0.5,markersize=2,label='PCR, total')
# ax2.plot(rnTime(curDates,meanWidth),100*rnMean(PCRposPct,meanWidth),'b',label='PCR, total, '+str(meanWidth)+' day mean')
# ax2.plot(curDates,100*AGposPct,'.-',color='xkcd:dark red',label='AG',linewidth=0.5,markersize=2)
# ax2.plot(rnTime(curDates,meanWidth),100*rnMean(AGposPct,meanWidth),color='xkcd:dark red',label='AG, '+str(meanWidth)+' day mean')
# ax2.plot(curDates,2*100*AGposPct,'.-',color='xkcd:red',label='AG, multiplied by 2',linewidth=0.5,markersize=2)
# ax2.plot(rnTime(curDates,meanWidth),2*100*rnMean(AGposPct,meanWidth),color='xkcd:red',label='AG, multiplied by 2, '+str(meanWidth)+' day mean')

# ax2.plot(curDates,100*np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),'.-',linewidth=0.5,markersize=2,color='xkcd:green',label='PCR, screening')
# ax2.plot(rnTime(curDates,meanWidth),rnMean(100*np.divide(foundThroughPCRScreening,numPCR*(1-screenRatio)),meanWidth),color='xkcd:green',label='PCR, screening, '+str(meanWidth)+' day mean')
# ax2.plot(curDates,6*100*np.divide(PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),numPCR),label='PCR positive, as neighbours')

# ax2.plot(rnTime(curDates,meanWidth),100*rnMean(PCRposPct,meanWidth),'b')
# ax2.plot(rnTime(curDates,meanWidth),rnMean(6*100*np.divide(PCRpos - (AGposPCRpos + np.multiply(PCRpos_nonAG,multFactor)),numPCR),7),':')


In [62]:
dfPCR.tail() 
curTested = dfPCR[dfPCR.Date.isin(curDates)].Tested
# curTested

In [63]:
# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)
fig,ax1 = plt.subplots(1,1,sharex=True)

meanWidth = 7

# ax1.plot(curDates,PCRpos,'.-',color='xkcd:rust',linewidth=0.5,markersize=2,label='PCR positive, total')
# ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos,meanWidth),color='xkcd:rust')
# ax1.plot(curDates,PCRpos_nonAG,'g.-',linewidth=0.5,markersize=2,label='PCR positive, due to only PCR')
# ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),'g')

# perTest = np.divide(PCRpos_nonAG,curTested)
normFactor = (100000/curTested)**(0.7)
posNorm = np.divide(PCRpos_nonAG,normFactor)
# ax1.plot(curDates,perTest,'.-',linewidth=0.5,markersize=2,label='PCR positive, due to only PCR, per test')
ax1.plot(curDates,posNorm,'b.-',linewidth=0.5,markersize=2,label='PCR positive, due to only PCR, normalized')
ax1.plot(rnTime(curDates,meanWidth),rnMean(posNorm,meanWidth),'b')


# ax1.plot(curDates,AGposPCRpos,'.-',color='xkcd:hot pink',linewidth=0.5,markersize=2,label='PCR positive, due to AG')
# ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRpos,meanWidth),color='xkcd:hot pink')

# ax1.legend(loc='center center',bbox_to_anchor = (0.5, 1.0))
ax1.legend(loc='upper left')
# ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
# ax3.grid()
# ax1.set_ylim(bottom=0,top=900)
# ax2.set_ylim(bottom=0)
# ax3.set_ylim(bottom=0)

ax1.set_ylabel('Count')


ax1.set_xlim([firstDate,lastDate-np.timedelta64(1,'D')])
plt.tight_layout()


# if saveFigures:
#     plt.savefig('figs/Viggo_Screening')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
df.tail()

,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
396,2021-05-09,270,267772,0.1,784,268556,12165562
397,2021-05-10,434,486496,0.1,1426,487922,12653484
398,2021-05-11,446,388566,0.1,1032,389598,13043082
399,2021-05-12,459,419549,0.1,1085,420634,13463716
400,2021-05-13,316,292677,0.1,858,293535,13757251


In [65]:
df2.columns
plt.figure(figsize=(10,10))
# curdf = df2[df2.Dato >= np.datetime64('2021-03-01')]
# plt.plot(curdf['AG_testede'],curdf['AGpos_PCRpos'],'*')
# curdf = df2[df2.Dato >= np.datetime64('2021-04-01')]
# plt.plot(curdf['AG_testede'],curdf['AGpos_PCRpos'],'.')
curdf = df2[df2.Dato >= np.datetime64('2021-03-01')]
plt.plot(curdf['AG_pos'],curdf['AGpos_PCRpos'],'*')
curdf = df2[df2.Dato >= np.datetime64('2021-04-01')]
plt.plot(curdf['AG_pos'],curdf['AGpos_PCRpos'],'.')
plt.ylim(bottom=0)
plt.xlim(left=0)
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Plejehjem

In [66]:
# dfPle = pd.read_csv(latestdir+'/plejehjem_ugeoversigt.csv',delimiter = ';',dtype=str)
# dfPle = dfPle.iloc[:-1] # Remove sum in last row
# for name in dfPle.columns:
#     dfPle[name] = pd.to_numeric(dfPle[name].astype(str).apply(lambda x: x.replace('.','')))

In [67]:
# dfPle.tail()

In [68]:
# pleCols = dfPle.columns
# pleCols

In [69]:
# pleDates = dfPle['År'].astype(str) +'-'+ dfPle['Uge'].astype(str)

In [70]:
# # fig,ax1 = plt.subplots(1,1)
# # fig,(ax1,ax2,ax3,ax4,ax5) = plt.subplots(5,1,sharex=True)
# fig,allAxes = plt.subplots(5,1,sharex=True,figsize=(16,14))

# for k in range(0,len(pleCols)-2):
#     curAx = allAxes[k]
    
#     curAx.plot(pleDates,dfPle[pleCols[k+2]],'k')
#     curAx.set_title(pleCols[k+2])
    
#     curAx.set_ylim(bottom=0)
    
#     curAx.grid()

# # ax1.plot(pleDates,dfPle[pleCols[2]])
# # ax2.plot(pleDates,dfPle[pleCols[3]])
# # ax3.plot(pleDates,dfPle[pleCols[4]])
# # ax4.plot(pleDates,dfPle[pleCols[5]])
# # ax5.plot(pleDates,dfPle[pleCols[6]])
# # ax5.set_ylabel()

# # ax1.set_ylim(bottom=0)
# # ax2.set_ylim(bottom=0)
# # ax3.set_ylim(bottom=0)
# # ax4.set_ylim(bottom=0)
# # ax5.set_ylim(bottom=0)
# # ax1.grid()
# # ax2.grid()
# # ax3.grid()
# # ax4.grid()
# # ax5.grid()

# # ax1.set_xticks(pleDates[::2])
# # # ax1.set_xticklabels(rotation=90)
# # plt.xticks(rotation=90)

# curAx.set_xticks(pleDates[::2])
# plt.xticks(rotation=90)

# plt.tight_layout()

# if saveFigures:
#     plt.savefig('figs/Plejehjem')

# Age stuff

In [71]:
dfAge = pd.read_csv(latestdir+'/Cases_by_age.csv',delimiter = ';',dtype=str)

In [72]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath


conf = []
test = []
posp = []

for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])
        curdf = pd.read_csv(latestdir+'/Cases_by_age.csv',delimiter = ';',dtype=str)
        conf.append(curdf.iloc[9,1])
        test.append(curdf.iloc[9,2])
        posp.append(curdf.iloc[9,3])
        # print(curdf)

In [73]:
# dfAge = pd.read_csv''/Cases_by_age.csv',delimiter = ';',dtype=str)
latestdir

dfAge = pd.read_csv('ssi_data\\SSI_data_2021-04-06\\Cases_by_age.csv',delimiter = ';',dtype=str)

In [74]:
dfAge

,Aldersgruppe,Antal_bekræftede_COVID-19,Antal_testede,Procent_positive
0,0-9,17.004,411.910,"4,1"
1,10-19,36.065,608.757,"5,9"
2,20-29,43.793,679.566,"6,4"
3,30-39,32.409,596.068,"5,4"
4,40-49,34.396,637.766,"5,4"
5,50-59,33.504,651.292,"5,1"
6,60-69,18.247,487.720,"3,7"
7,70-79,11.219,369.509,"3,0"
8,80-89,5.850,140.967,"4,1"
9,90+,1.830,29.111,"6,3"


In [75]:
# dfDeaths = pd.read_excel("C://Users//rakrpe//Documents//PandemiX//GithubRepos//PandemiX//DanskeData//DeathsAgeDK//Deaths.xlsx")

In [76]:
dfDeaths = pd.read_excel('DeathsAgeDK/Deaths.xlsx')

In [77]:
dfDeaths = dfDeaths.transpose()

In [78]:
dfDeaths.columns = dfDeaths.iloc[0]

In [79]:
dfDeaths = dfDeaths.drop(['Aldersgruppe']) 

In [80]:
# plt.figure()
# plt.plot(dfDeaths)


curDates =  pd.to_datetime(dfDeaths.index,format='%d_%m_%Y')

In [82]:
# plt.figure()
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)

# for colNames in dfDeaths.columns[:-1]:
for colNames in dfDeaths.columns:
    # plt.plot(curDates,dfDeaths['0-9'])
    ax1.plot(curDates,dfDeaths[colNames],label=colNames)
    ax2.plot(curDates[0:-1],np.diff(dfDeaths[colNames]),label=colNames)
    ax3.plot(curDates[0:-1],np.diff(dfDeaths[colNames]),label=colNames)

ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()
ax2.grid()
ax3.grid()

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax3.set_ylim(bottom=0,top=10)

# ax1.set_xlim(left=curDates[0],right=date.today())
ax1.set_xlim(right=date.today())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d %b\n%Y'))

ax1.set_ylabel('Kumulative dødsfald')
ax2.set_ylabel('Dødsfald')
ax3.set_ylabel('Dødsfald')

plt.tight_layout()

# if saveFigures:
#     fig.savefig('figs/DeathsByAge')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [83]:
# plt.figure()
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,ax1 = plt.subplots(1,1,sharex=True)

curVals = np.zeros(curDates[0:-1].shape)
for colNames in dfDeaths.columns:

    curVals += np.diff(dfDeaths[colNames])
    ax1.plot(curDates[0:-1],curVals)

    # plt.plot(curDates,dfDeaths['0-9'])
    # ax1.plot(curDates,dfDeaths[colNames],label=colNames)

    # ax2.plot(curDates[0:-1],np.diff(dfDeaths[colNames]),label=colNames)
    # ax3.plot(curDates[0:-1],np.diff(dfDeaths[colNames]),label=colNames)

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.grid()

ax1.set_ylim(bottom=0)

# ax1.set_xlim(left=curDates[0],right=date.today())
ax1.set_xlim(right=date.today())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d %b\n%Y'))

ax1.set_ylabel('Kumulative dødsfald')
# ax2.set_ylabel('Dødsfald')
# ax3.set_ylabel('Dødsfald')

plt.tight_layout()

# if saveFigures:
#     fig.savefig('figs/DeathsByAge')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: ufunc 'add' output (typecode 'O') could not be coerced to provided output parameter (typecode 'd') according to the casting rule ''same_kind''

In [ ]:
# np.diff(dfDeaths[colNames])
# curVals
dfDeaths

In [ ]:
dfAge

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)


ax1.plot(curDates,dfDeaths['60-69'])

plt.tight_layout()

In [ ]:
curDates